In [1]:
import sys
sys.path.append('../../test_bed')

In [2]:
sys.path

['c:\\Users\\USER\\Quick_Ref\\Programming\\Python\\Python 중급\\실용 파이썬 프로그래밍',
 'c:\\Users\\USER\\anaconda3\\python39.zip',
 'c:\\Users\\USER\\anaconda3\\DLLs',
 'c:\\Users\\USER\\anaconda3\\lib',
 'c:\\Users\\USER\\anaconda3',
 '',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages\\win32',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\Users\\USER\\anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\USER\\.ipython',
 '../../test_bed']

In [3]:
import fileparse

with open('../../data/portfolio.csv') as lines:
    portdicts = fileparse.parse_csv(lines, select=['name', 'shares', 'price'], types=[str, int, float])
portdicts    

[{'name': 'AA', 'shares': 100, 'price': 32.2},
 {'name': 'IBM', 'shares': 50, 'price': 91.1},
 {'name': 'CAT', 'shares': 150, 'price': 83.44},
 {'name': 'MSFT', 'shares': 200, 'price': 51.23},
 {'name': 'GE', 'shares': 95, 'price': 40.37},
 {'name': 'MSFT', 'shares': 50, 'price': 65.1},
 {'name': 'IBM', 'shares': 100, 'price': 70.44}]

In [31]:
%%writefile ../../test_bed/stock.py
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
    def cost(self):
        return self.shares * self.price
        
    def sell(self, shares):
        self.shares -= shares            


Writing ../../test_bed/stock.py


In [34]:
a = Stock('GOOG', 100, 490.10)
b = Stock('AAPL', 50, 122.34)
b.name, b.shares, b.price        

c = Stock('IBM', 75, 91.75)
c.name, b.shares, b.price        
c.shares * c.price

stocks = [a, b, c]
stocks
for s in stocks:
    print(f'{s.name} {s.shares} {s.price}')
    
a.cost()    
a.sell(25)
a.shares

import fileparse
with open('../../data/portfolio.csv') as lines:
    portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str, int, float])
    portfolio = [ Stock(d['name'], d['shares'], d['price']) for d in portdicts ]
portfolio    

# sum([ s.shares * s.price for s in portfolio ])


GOOG 100 490.1
AAPL 50 122.34
IBM 75 91.75


In [35]:
%%writefile ../../test_bed/prices.py
class Prices:
    def __init__(self, name, price):
        self.name = name
        self.price = price

Writing ../../test_bed/prices.py


In [4]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    headers = ('Name','Shares','Price','Change')
    print('%10s %10s %10s %10s' % headers)
    print(('-'*10 + ' ')*len(headers))
    for row in reportdata:
        print('%10s %10d %10.2f %10.2f' % row)

def portfolio_report(portfoliofile, pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data
    report = make_report_data(portfolio, prices)

    # Print it out
    print_report(report)

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv)

Overwriting ../../test_bed/report.py


In [39]:
%%writefile ../../test_bed/pcost.py
# pcost.py

import report

def portfolio_cost(filename):
    '''
    Computes the total cost (shares*price) of a portfolio file
    '''
    portfolio = report.read_portfolio(filename)
    return sum([s.shares * s.price for s in portfolio])

def main(args):
    if len(args) != 2:
        raise SystemExit('Usage: %s portfoliofile' % args[0])
    filename = args[1]
    print('Total cost:', portfolio_cost(filename))

if __name__ == '__main__':
    import sys
    main(sys.argv)

Overwriting ../../test_bed/pcost.py


In [1]:
import sys
sys.path.append('../../test_bed')

In [2]:
import pcost

pcost.portfolio_cost('../../data/portfolio.csv')

[report.py]<read_portfolio()> Return to Object


44671.15

In [3]:
import report
report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv')

[report.py]<read_portfolio()> Return to Object
      Name     Shares      Price     Change
---------- ---------- ---------- ---------- 
        AA        100       9.22     -22.98
       IBM         50     106.28      15.18
       CAT        150      35.46     -47.98
      MSFT        200      20.89     -30.34
        GE         95      13.48     -26.89
      MSFT         50      20.89     -44.21
       IBM        100     106.28      35.84


# 4.2 상속

In [ ]:
# 도입
# ----
# 상속은 기존 객체를 특수화하는 데 사용된다.

# class Parent:
#     ...

# class Child(Parent):
#     ...
# 위의 Child 클래스와 같은 것을 파생 클래스(derived class) 또는 하위 클래스(subclass)라 한다. 
# 위의 Parent 클래스는 기본 클래스(base class) 또는 상위 클래스(superclass)라 한다. 
# class Child(Parent):에서 클래스명 뒤의 ()에 Parent를 지정했다.



# 확장(Extending)
# ---------------
# 상속을 통해 기존 클래스를 취하여 다음과 같은 일을 할 수 있다.

# 새로운 메서드를 추가
# 기존 메서드 일부를 재정의(redefine)
# 인스턴스에 새로운 어트리뷰트를 추가
# 그리하여 기존 코드를 확장하게 된다.

# 예시
# 이 클래스에서 시작한다고 하자.

# class Stock:
#     def __init__(self, name, shares, price):
#         self.name = name
#         self.shares = shares
#         self.price = price

#     def cost(self):
#         return self.shares * self.price

#     def sell(self, nshares):
#         self.shares -= nshares

# 상속을 통해 이것의 어느 부분이든 변경할 수 있다.



# 새 메서드를 추가
# ---------------
# class MyStock(Stock):
#     def panic(self):
#         self.sell(self.shares)

# 용례:
# >>> s = MyStock('GOOG', 100, 490.1)
# >>> s.sell(25)
# >>> s.shares
# 75
# >>> s.panic()
# >>> s.shares
# 0
# >>>



# 기존 메서드를 재정의
# -------------------
# class MyStock(Stock):
#     def cost(self):
#         return 1.25 * self.shares * self.price

# 용례:
# >>> s = MyStock('GOOG', 100, 490.1)
# >>> s.cost()
# 61262.5
# >>>

# 새 메서드는 기존 것을 대체한다. 다른 메서드는 영향을 받지 않는다. 굉장하다.

In [6]:
import sys
sys.path.append('../../test_bed')

In [10]:
# %load ../../test_bed/stock.py
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
    def cost(self):
        return self.shares * self.price
        
    def sell(self, shares):
        self.shares -= shares            


In [18]:
#import stock

class MyStock(Stock):
    def panic(self):
        self.sell(self.shares)
    
    def cost(self):
        return 1.25 * self.shares * self.price

s = MyStock('GOOG', 100, 490.1)
s.name, s.shares, s.price

s.cost()

    

61262.5

## 오버라이딩(Overriding)

In [ ]:
# 때로는 클래스가 기존 메서드를 확장하되, 원래 구현을 재정의에 포함하고 싶을 수 있다. 이를 위해 super()를 사용한다.

# class Stock:
#     ...
#     def cost(self):
#         return self.shares * self.price
#     ...

# class MyStock(Stock):
#     def cost(self):
#         # `super`에 대한 호출을 확인
#         actual_cost = super().cost()
#         return 1.25 * actual_cost

# 이전 버전을 호출하기 위해 super()를 사용한다.

# 주의: 파이썬 2는 이것보다 구문이 복잡하다.

# actual_cost = super(MyStock, self).cost()

In [19]:
class MyStock(Stock):
    def panic(self):
        self.sell(self.shares)
    
    def cost(self):
        #'super'에 대한 호출을 확인
        actual_cost = super().cost()
        return 1.25 * actual_cost

s = MyStock('GOOG', 100, 490.1)
s.cost()

    

61262.5

## __init__와 상속

In [ ]:
# __init__와 상속
# ---------------
# __init__를 재정의하려면 부모를 초기화해야 한다.

# class Stock:
#     def __init__(self, name, shares, price):
#         self.name = name
#         self.shares = shares
#         self.price = price

# class MyStock(Stock):
#     def __init__(self, name, shares, price, factor):
#         # `super`와 `__init__`에 대한 호출을 확인
#         super().__init__(name, shares, price)
#         self.factor = factor

#     def cost(self):
#         return self.factor * super().cost()

# super에 __init__() 메서드를 호출해야 한다. 이것이 앞에서 본 것과 같이 이전 버전을 호출하는 방법이다

In [20]:
class MyStock(Stock):
    def __init__(self, name, shares, price, factor):
        #super와 __init__ 에 대한 호출을 확인
        super().__init__(name, shares, price)
        self.factor = factor
        
    def panic(self):
        self.sell(self.shares)
    
    def cost(self):
        #'super'에 대한 호출을 확인
        actual_cost = super().cost()
        return self.factor * actual_cost

s = MyStock('GOOG', 100, 490.1, 1.25)
s.cost()

    

61262.5

## 상속을 사용하기

In [ ]:
# 상속을 사용하기
# --------------
# 상속은 관련 객체를 조직화하는 데 사용되곤 한다.

# class Shape:
#     ...

# class Circle(Shape):
#     ...

# class Rectangle(Shape):
#     ...

# 논리적 계층 구조나 분류법을 생각해 보라. 
# 그렇지만 좀 더 일반적이고도 실용적인 용도는 재사용 가능 혹은 확장 가능한 코드를 만드는 것이다. 
# 예를 들어, 프레임워크에 정의된 기본 클래스를 당신이 커스터마이즈할 수 있다.

# class CustomHandler(TCPHandler):
#     def handle_request(self):
#         ...
#         # 커스텀 처리

# 기본 클래스는 일반 목적 코드를 포함한다. 당신의 클래스는 기본 클래스를 상속해 특수한 부분을 커스터마이즈한다.


## "is a" 관계

In [ ]:
# "is a" 관계
# -----------
# 상속은 타입 간의 관계를 설정한다.

# class Shape:
#     ...

# class Circle(Shape):
#     ...
# 객체 인스턴스를 확인하라.

# >>> c = Circle(4.0)
# >>> isinstance(c, Shape)
# True

# 중요: 부모 클래스의 인스턴스를 다루는 코드는 자식 클래스의 인스턴스에 대해서도 작동하는 것이 이상적이다.

In [21]:
s = MyStock('GOOG', 100, 490.1, 1.25)
isinstance(s, Stock)


True

## object 기본 클래스

In [ ]:
# object 기본 클래스
# -----------------
# 만약 클래스에 부모가 없으면 object를 기본으로 삼을 수 있다.

# class Shape(object):
#     ...
# 파이썬에서 object는 모든 객체의 부모다.

# *참고: 기술적으로 이것이 필요하지는 않지만, 파이썬 2에서 필요했기 때문에 남겨둔 경우가 많다. 
#       object를 생략하더라도 암시적으로 상속한다.

## 다중 상속

In [ ]:
# 다중 상속
# --------
# 여러 클래스로부터 상속하도록 클래스에 정의할 수 있다.

# class Mother:
#     ...

# class Father:
#     ...

# class Child(Mother, Father):
#     ...
# Child 클래스는 Mother와 Father의 특징을 모두 상속받는다. 
# 다중 상속은 세부적으로 들어가면 골치가 아파진다. 
# 정확히 이해하기 전에는 사용하지 마라. 다음 섹션에서 추가 정보를 제공하겠지만, 이 코스에서 다중 상속을 사용할 일은 없다.

## 연습 문제

In [ ]:
# 상속의 주 용도는 다양한 방법으로 확장 또는 커스터마이즈된 코드를 작성하는 것에 있다 (특히 라이브러리나 프레임워크에서). 
# 설명을 위해 report.py 프로그램의 print_report() 함수를 생각해 보자. 그것은 다음과 같이 보일 것이다.

# def print_report(reportdata):
#     '''
#     (name, shares, price, change) 튜플의 리스트로부터 보기 좋게 포매팅한 테이블을 프린팅.
#     '''
#     headers = ('Name','Shares','Price','Change')
#     print('%10s %10s %10s %10s' % headers)
#     print(('-'*10 + ' ')*len(headers))
#     for row in reportdata:
#         print('%10s %10d %10.2f %10.2f' % row)
# report 프로그램을 실행하면 다음과 같이 출력한다.

# >>> import report
# >>> report.portfolio_report('Data/portfolio.csv', 'Data/prices.csv')
#       Name     Shares      Price     Change
# ---------- ---------- ---------- ----------
#         AA        100       9.22     -22.98
#        IBM         50     106.28      15.18
#        CAT        150      35.46     -47.98
#       MSFT        200      20.89     -30.34
#         GE         95      13.48     -26.89
#       MSFT         50      20.89     -44.21
#        IBM        100     106.28      35.84

## 연습 문제 4.5: 확장성 문제

In [ ]:
# print_report() 함수가 일반 텍스트, HTML, CSV, XML 같이 다양한 출력 포맷을 지원하게 수정하고 싶다고 하자. 
# 그렇게 하려고 모든 일을 처리하는 아주 큰 함수를 작성하려고 시도할 수 있다. 
# 하지만 그렇게 했다가는 유지보수를 하기 힘들만큼 지저분해질 수 있다. 이때가 상속을 사용하기에 아주 좋은 기회다.

# 시작하려면 테이블을 생성하는 단계에 집중하자. 
# 테이블의 첫 행에는 헤더가 있다. 그 후, 테이블 데이터 행들이 나타난다. 
# 다음 단계를 따라 자체 클래스에 집어넣자. 
# 다음과 같은 클래스를 정의하는 tableformat.py 파일을 작성한다.

# # tableformat.py

# class TableFormatter:
#     def headings(self, headers):
#         '''
#         테이블 헤딩을 반환.
#         '''
#     raise NotImplementedError()

#     def row(self, rowdata):
#         '''
#         테이블 데이터의 단일 행을 반환.
#         '''
#     raise NotImplementedError()

# 이 클래스는 아무 일도 하지 않지만, 곧 정의할 추가 클래스의 설계 사양과 역할을 한다. 
# 이러한 클래스를 "추상 기본 클래스(abstract base class)"라고 부른다.

# 입력으로 받은 TableFormatter 객체의 메서드를 호출해 출력을 생성하도록 print_report() 함수를 수정하자. 예를 들면 다음과 같다.

# # report.py
# ...

# def print_report(reportdata, formatter):
#     '''
#     (name, shares, price, change) 튜플의 리스트로부터 보기 좋게 포매팅한 테이블을 프린팅.
#     '''
#     formatter.headings(['Name','Shares','Price','Change'])
#     for name, shares, price, change in reportdata:
#         rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
#         formatter.row(rowdata)

# print_report()에 인자를 추가했으므로, portfolio_report() 함수도 수정해야 한다. TableFormatter를 생성하게 변경한다.

# # report.py

# import tableformat

# ...
# def portfolio_report(portfoliofile, pricefile):
#     '''
#     주어진 포트폴리오와 가격 데이터 파일을 가지고 주식 보고서를 작성.
#     '''
#     # 데이터 파일 읽기
#     portfolio = read_portfolio(portfoliofile)
#     prices = read_prices(pricefile)

#     # 보고서 데이터 생성
#     report = make_report_data(portfolio, prices)

#     # 프린트
#     formatter = tableformat.TableFormatter()
#     print_report(report, formatter)
# 새 코드를 실행하자.

# >>> ================================ RESTART ================================
# >>> import report
# >>> report.portfolio_report('Data/portfolio.csv', 'Data/prices.csv')
# ... 충돌 ...
# 곧바로 NotImplementedError 예외를 내며 충돌한다. 썩 즐겁지는 않지만, 이것은 예상했던 바다. 다음 부분으로 이어 가자.

In [3]:
%%writefile ../../test_bed/tableformat.py
# tableformat.py

class TableFormatter:
    def headings(self, headers):
        '''
        테이블 헤딩을 반환.
        '''
        raise NotImplementedError()

    def row(self, rowdata):
        '''
        테이블 데이터의 단일 행을 반환.
        '''
        raise NotImplementedError()

class TextTableFormatter(TableFormatter):
    def headings(self, headers):
        for h in headers:
            print(f'{h:>10s}', end=' ')
        print()
        print(('-'*10 + ' ') * len(headers))
            
    def row(self, rowdata):
        for d in rowdata:
            print(f'{d:>10s}', end=' ')
        print()


Overwriting ../../test_bed/tableformat.py


In [16]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    formatter = tableformat.TextTableFormatter()
    print_report(report, formatter)

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv)


Overwriting ../../test_bed/report.py


In [1]:
import sys
sys.path.append('../../test_bed')

In [2]:
import report
report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv')


[report.py]<read_portfolio()> Return to Object
      Name     Shares      Price     Change 
---------- ---------- ---------- ---------- 
        AA        100       9.22     -22.98 
       IBM         50     106.28      15.18 
       CAT        150      35.46     -47.98 
      MSFT        200      20.89     -30.34 
        GE         95      13.48     -26.89 
      MSFT         50      20.89     -44.21 
       IBM        100     106.28      35.84 


## 연습 문제 4.6: 상속을 통해 다른 출력을 생성하기

In [ ]:
# (a) 부분에서 정의한 TableFormatter 클래스는 상속을 통해 확장할 의도를 가지고 정의했다. 
# 사실, 그것이 아이디어의 전부다. 다음과 같이 TextTableFormatter 클래스를 정의한다.

# # tableformat.py
# ...
# class TextTableFormatter(TableFormatter):
#     '''
#     테이블을 일반 텍스트 포맷으로 출력
#     '''
#     def headings(self, headers):
#         for h in headers:
#             print(f'{h:>10s}', end=' ')
#         print()
#         print(('-'*10 + ' ')*len(headers))

#     def row(self, rowdata):
#         for d in rowdata:
#             print(f'{d:>10s}', end=' ')
#         print()
# portfolio_report() 함수를 다음과 같이 수정하고 사용해 보자.

# # report.py
# ...
# def portfolio_report(portfoliofile, pricefile):
#     '''
#     주어진 포트폴리오와 가격 데이터 파일을 가지고 주식 보고서를 작성.
#     '''
#     # 데이터 파일 읽기
#     portfolio = read_portfolio(portfoliofile)
#     prices = read_prices(pricefile)

#     # 보고서 데이터 생성
#     report = make_report_data(portfolio, prices)

#     # 프린트
#     formatter = tableformat.TextTableFormatter()
#     print_report(report, formatter)
# 이전과 같은 출력을 생성해야 한다.

# >>> ================================ RESTART ================================
# >>> import report
# >>> report.portfolio_report('Data/portfolio.csv', 'Data/prices.csv')
#       Name     Shares      Price     Change
# ---------- ---------- ---------- ----------
#         AA        100       9.22     -22.98
#        IBM         50     106.28      15.18
#        CAT        150      35.46     -47.98
#       MSFT        200      20.89     -30.34
#         GE         95      13.48     -26.89
#       MSFT         50      20.89     -44.21
#        IBM        100     106.28      35.84
# >>>


In [6]:
%%writefile ../../test_bed/tableformat.py
# tableformat.py

class TableFormatter:
    def headings(self, headers):
        '''
        테이블 헤딩을 반환.
        '''
        raise NotImplementedError()

    def row(self, rowdata):
        '''
        테이블 데이터의 단일 행을 반환.
        '''
        raise NotImplementedError()

class TextTableFormatter(TableFormatter):
    def headings(self, headers):
        for h in headers:
            print(f'{h:>10s}', end=' ')
        print()
        print(('-'*10 + ' ') * len(headers))
            
    def row(self, rowdata):
        for d in rowdata:
            print(f'{d:>10s}', end=' ')
        print()

In [7]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    formatter = tableformat.TextTableFormatter()
    print_report(report, formatter)

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv)


In [11]:
import report
report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv')


[report.py]<read_portfolio()> Return to Object
      Name     Shares      Price     Change 
---------- ---------- ---------- ---------- 
        AA        100       9.22     -22.98 
       IBM         50     106.28      15.18 
       CAT        150      35.46     -47.98 
      MSFT        200      20.89     -30.34 
        GE         95      13.48     -26.89 
      MSFT         50      20.89     -44.21 
       IBM        100     106.28      35.84 


In [ ]:
# 하지만, 출력을 다른 것으로 바꾸자. 출력을 CSV 포맷으로 출력하는 새로운 클래스 CSVTableFormatter를 정의한다.

# # tableformat.py
# ...
# class CSVTableFormatter(TableFormatter):
#     '''
#     포트폴리오 데이터를 CSV 포맷으로 출력.
#     '''
#     def headings(self, headers):
#         print(','.join(headers))

#     def row(self, rowdata):
#         print(','.join(rowdata))
# 메인 프로그램을 다음과 같이 수정한다.


# def portfolio_report(portfoliofile, pricefile):
#     '''
#     주어진 포트폴리오와 가격 데이터 파일을 가지고 주식 보고서를 작성.
#     '''
#     # 데이터 파일 읽기
#     portfolio = read_portfolio(portfoliofile)
#     prices = read_prices(pricefile)

#     # 보고서 데이터 생성
#     report = make_report_data(portfolio, prices)

#     # 프린트
#     formatter = tableformat.CSVTableFormatter()
#     print_report(report, formatter)

In [4]:
%%writefile ../../test_bed/tableformat.py
# tableformat.py

class TableFormatter:
    def headings(self, headers):
        '''
        테이블 헤딩을 반환.
        '''
        raise NotImplementedError()

    def row(self, rowdata):
        '''
        테이블 데이터의 단일 행을 반환.
        '''
        raise NotImplementedError()

class TextTableFormatter(TableFormatter):
    def headings(self, headers):
        for h in headers:
            print(f'{h:>10s}', end=' ')
        print()
        print(('-'*10 + ' ')*len(headers))

    def row(self, rowdata):
        for d in rowdata:
            print(f'{d:>10s}', end=' ')
        print()

class CSVTableFormatter(TableFormatter):
    def headings(self, headers):
        print(','.join(headers))
            
    def row(self, rowdata):
        print(','.join(rowdata))


Overwriting ../../test_bed/tableformat.py


In [5]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    #formatter = tableformat.TextTableFormatter()
    formatter = tableformat.CSVTableFormatter()
    print_report(report, formatter)

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv)


Overwriting ../../test_bed/report.py


In [ ]:

# 이제 다음과 같은 CSV 출력을 볼 수 있을 것이다.

# >>> ================================ RESTART ================================
# >>> import report
# >>> report.portfolio_report('Data/portfolio.csv', 'Data/prices.csv')
# Name,Shares,Price,Change
# AA,100,9.22,-22.98
# IBM,50,106.28,15.18
# CAT,150,35.46,-47.98
# MSFT,200,20.89,-30.34
# GE,95,13.48,-26.89
# MSFT,50,20.89,-44.21
# IBM,100,106.28,35.84

In [1]:
import sys
sys.path.append('../../test_bed')

In [2]:
import report
report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv')


[report.py]<read_portfolio()> Return to Object
Name,Shares,Price,Change
AA,100,9.22,-22.98
IBM,50,106.28,15.18
CAT,150,35.46,-47.98
MSFT,200,20.89,-30.34
GE,95,13.48,-26.89
MSFT,50,20.89,-44.21
IBM,100,106.28,35.84


In [ ]:

# 마찬가지로, 다음과 같은 출력을 생성하는 HTMLTableFormatter 클래스를 정의한다.

# <tr><th>Name</th><th>Shares</th><th>Price</th><th>Change</th></tr>
# <tr><td>AA</td><td>100</td><td>9.22</td><td>-22.98</td></tr>
# <tr><td>IBM</td><td>50</td><td>106.28</td><td>15.18</td></tr>
# <tr><td>CAT</td><td>150</td><td>35.46</td><td>-47.98</td></tr>
# <tr><td>MSFT</td><td>200</td><td>20.89</td><td>-30.34</td></tr>
# <tr><td>GE</td><td>95</td><td>13.48</td><td>-26.89</td></tr>
# <tr><td>MSFT</td><td>50</td><td>20.89</td><td>-44.21</td></tr>
# <tr><td>IBM</td><td>100</td><td>106.28</td><td>35.84</td></tr>

# CSVTableFormatter 객체 대신 HTMLTableFormatter 객체를 생성하게 메인 프로그램을 수정해 코드를 테스트하자.

In [3]:
%%writefile ../../test_bed/tableformat.py
# tableformat.py

class TableFormatter:
    def headings(self, headers):
        '''
        테이블 헤딩을 반환.
        '''
        raise NotImplementedError()

    def row(self, rowdata):
        '''
        테이블 데이터의 단일 행을 반환.
        '''
        raise NotImplementedError()

class TextTableFormatter(TableFormatter):
    def headings(self, headers):
        for h in headers:
            print(f'{h:>10s}', end=' ')
        print()
        print(('-'*10 + ' ')*len(headers))

    def row(self, rowdata):
        for d in rowdata:
            print(f'{d:>10s}', end=' ')
        print()

class CSVTableFormatter(TableFormatter):
    def headings(self, headers):
        print(','.join(headers))
            
    def row(self, rowdata):
        print(','.join(rowdata))
        
class HTMLTableFormatter(TableFormatter):
    def headings(self, headers):
        print('<tr>', end='')
        for h in headers:
            print(f'<th>{h}</th>', end='')
        print('</tr>')            
                    
    def row(self, rowdata):
        print('<tr>', end='')
        for r in rowdata:
            print(f'<th>{r}</th>', end='')
        print('</tr>')            

Overwriting ../../test_bed/tableformat.py


In [7]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    #formatter = tableformat.TextTableFormatter()
    #formatter = tableformat.CSVTableFormatter()
    formatter = tableformat.HTMLTableFormatter()
    print_report(report, formatter)

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv)


Overwriting ../../test_bed/report.py


In [1]:
import sys
sys.path.append('../../test_bed')

In [2]:
import report
report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv')


[report.py]<read_portfolio()> Return to Object
<tr><th>Name</th><th>Shares</th><th>Price</th><th>Change</th></tr>
<tr><th>AA</th><th>100</th><th>9.22</th><th>-22.98</th></tr>
<tr><th>IBM</th><th>50</th><th>106.28</th><th>15.18</th></tr>
<tr><th>CAT</th><th>150</th><th>35.46</th><th>-47.98</th></tr>
<tr><th>MSFT</th><th>200</th><th>20.89</th><th>-30.34</th></tr>
<tr><th>GE</th><th>95</th><th>13.48</th><th>-26.89</th></tr>
<tr><th>MSFT</th><th>50</th><th>20.89</th><th>-44.21</th></tr>
<tr><th>IBM</th><th>100</th><th>106.28</th><th>35.84</th></tr>


## 연습 문제 4.7: 다형성

In [ ]:
# 객체 지향 프로그래밍의 주된 특징은 기존 코드를 변경할 필요 없이 객체를 프로그램에 끼워넣을 수 있다는 것이다. 
# 예를 들어, TableFormatter 객체를 사용할 것으로 예상되는 프로그램을 작성했다고 하면, 
# 어떤 종류의 TableFormatter를 주는지에 관계없이 작동할 것이다. 이러한 행위를 '다형성(polymorphism)'이라 한다.

# 한 가지 잠재적인 문제는 사용자가 원하는 포매터를 선택하게 하는 방법을 알아내는 것이다. 
# TextTableFormatter 같은 클래스명을 직접 사용하게 하는 것은 종종 성가시다. 
# 따라서 단순한 접근을 고려할 수 있다. 다음과 같이 코드에 if 문을 포함할 수 있을 것이다.

# def portfolio_report(portfoliofile, pricefile, fmt='txt'):
#     '''
#     주어진 포트폴리오와 가격 데이터 파일을 가지고 주식 보고서를 작성.
#     '''
#     # 데이터 파일 읽기
#     portfolio = read_portfolio(portfoliofile)
#     prices = read_prices(pricefile)

#     # 보고서 데이터 생성
#     report = make_report_data(portfolio, prices)

#     # 프린트
#     if fmt == 'txt':
#         formatter = tableformat.TextTableFormatter()
#     elif fmt == 'csv':
#         formatter = tableformat.CSVTableFormatter()
#     elif fmt == 'html':
#         formatter = tableformat.HTMLTableFormatter()
#     else:
#         raise RuntimeError(f'Unknown format {fmt}')
#     print_report(report, formatter)

# 이 코드에서, 사용자는 'txt'나 'csv' 같이 단순화된 이름을 지정함으로써 포맷을 선택한다. 
# 하지만, portfolio_report() 함수에 커다란 if 문을 넣는 것이 최선인가? 
# 그 코드를 다른 범용 함수로 옮기는 것이 나을 것이다.

# 'txt', 'csv', 'html' 같은 출력 이름에 따라 사용자가 포매터를 생성하는 create_formatter(name) 함수를 tableformat.py 파일에 추가한다. 

# portfolio_report()를 다음과 같이 수정하자.

# def portfolio_report(portfoliofile, pricefile, fmt='txt'):
#     '''
#     주어진 포트폴리오와 가격 데이터 파일을 가지고 주식 보고서를 작성.
#     '''
#     # 데이터 파일 읽기
#     portfolio = read_portfolio(portfoliofile)
#     prices = read_prices(pricefile)

#     # 보고서 데이터 생성
#     report = make_report_data(portfolio, prices)

#     # 프린트
#     formatter = tableformat.create_formatter(fmt)
#     print_report(report, formatter)
# 여러 포맷으로 함수를 호출해 함수가 제대로 작동하는지 확인해 보자.

In [8]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile, fmt='txt'):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    #formatter = tableformat.TextTableFormatter()
    #formatter = tableformat.CSVTableFormatter()

    formatter = tableformat.create_formatter(fmt)
    print_report(report, formatter)
    

def main(args):
    if len(args) != 3:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2])

if __name__ == '__main__':
    import sys
    main(sys.argv, sys)


Overwriting ../../test_bed/report.py


In [8]:
%%writefile ../../test_bed/tableformat.py
# tableformat.py

class TableFormatter:
    def headings(self, headers):
        '''
        테이블 헤딩을 반환.
        '''
        raise NotImplementedError()

    def row(self, rowdata):
        '''
        테이블 데이터의 단일 행을 반환.
        '''
        raise NotImplementedError()
    

class TextTableFormatter(TableFormatter):
    def headings(self, headers):
        for h in headers:
            print(f'{h:>10s}', end=' ')
        print()
        print(('-'*10 + ' ')*len(headers))

    def row(self, rowdata):
        for d in rowdata:
            print(f'{d:>10s}', end=' ')
        print()
        

class CSVTableFormatter(TableFormatter):
    def headings(self, headers):
        print(','.join(headers))
            
    def row(self, rowdata):
        print(','.join(rowdata))
        
        
class HTMLTableFormatter(TableFormatter):
    def headings(self, headers):
        print('<tr>', end='')
        for h in headers:
            print(f'<th>{h}</th>', end='')
        print('</tr>')            
                    
    def row(self, rowdata):
        print('<tr>', end='')
        for r in rowdata:
            print(f'<th>{r}</th>', end='')
        print('</tr>')            


def create_formatter(fmt):
    if fmt == 'txt':
        formatter = TextTableFormatter()
    elif fmt == 'csv':
        formatter = CSVTableFormatter()
    elif fmt == 'html':
        formatter = HTMLTableFormatter()
    else:
        raise RuntimeError(f'Unknown format {fmt}')
    
    return formatter

Overwriting ../../test_bed/tableformat.py


In [1]:
import sys
sys.path.append('../../test_bed')

In [3]:
import report

report.portfolio_report('../../data/portfolio.csv', '../../data/prices.csv', fmt='html')

[report.py]<read_portfolio()> Return to Object
<tr><th>Name</th><th>Shares</th><th>Price</th><th>Change</th></tr>
<tr><th>AA</th><th>100</th><th>9.22</th><th>-22.98</th></tr>
<tr><th>IBM</th><th>50</th><th>106.28</th><th>15.18</th></tr>
<tr><th>CAT</th><th>150</th><th>35.46</th><th>-47.98</th></tr>
<tr><th>MSFT</th><th>200</th><th>20.89</th><th>-30.34</th></tr>
<tr><th>GE</th><th>95</th><th>13.48</th><th>-26.89</th></tr>
<tr><th>MSFT</th><th>50</th><th>20.89</th><th>-44.21</th></tr>
<tr><th>IBM</th><th>100</th><th>106.28</th><th>35.84</th></tr>


## 연습 문제 4.8: 모두 합치기

In [ ]:
# portfolio_report() 함수가 출력 포맷을 지정하는 선택적 인자를 받게 report.py 프로그램을 수정하라. 예:

# >>> report.portfolio_report('Data/portfolio.csv', 'Data/prices.csv', 'txt')
#       Name     Shares      Price     Change
# ---------- ---------- ---------- ----------
#         AA        100       9.22     -22.98
#        IBM         50     106.28      15.18
#        CAT        150      35.46     -47.98
#       MSFT        200      20.89     -30.34
#         GE         95      13.48     -26.89
#       MSFT         50      20.89     -44.21
#        IBM        100     106.28      35.84
# >>>
# 명령행에서 포맷을 지정하게 메인 프로그램을 수정한다.

# bash $ python3 report.py Data/portfolio.csv Data/prices.csv csv
# Name,Shares,Price,Change
# AA,100,9.22,-22.98
# IBM,50,106.28,15.18
# CAT,150,35.46,-47.98
# MSFT,200,20.89,-30.34
# GE,95,13.48,-26.89
# MSFT,50,20.89,-44.21
# IBM,100,106.28,35.84
# bash $

In [7]:
%%writefile ../../test_bed/report.py
# report.py

import fileparse
import stock
import prices
import tableformat

def read_portfolio(filename):
    '''
    Read a stock portfolio file into a list of dictionaries with keys
    name, shares, and price.
    '''
    with open(filename) as lines:
        # return fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])

        # Convert Dictionary to Object     
        portdicts = fileparse.parse_csv(lines, select=['name','shares','price'], types=[str,int,float])
        print('[report.py]<read_portfolio()> Return to Object')
        return [ stock.Stock(d['name'], d['shares'], d['price']) for d in portdicts ]

def read_prices(filename):
    '''
    Read a CSV file of price data into a dict mapping names to prices.
    '''
    with open(filename) as lines:
        return dict(fileparse.parse_csv(lines, types=[str,float], has_headers=False))

        # Convert Dictionary to Object     
        # portdicts = fileparse.parse_csv(lines, types=[str,float], has_headers=False)
        # return [ prices.Prices(d.name, d.price) for d in portdicts ]


def make_report_data(portfolio,prices):
    '''
    Make a list of (name, shares, price, change) tuples given a portfolio list
    and prices dictionary.
    '''
    rows = []
    for stock in portfolio:
        current_price = prices[stock.name]
        change = current_price - stock.price
        summary = (stock.name, stock.shares, current_price, change)
        rows.append(summary)
    return rows

def print_report(reportdata, formatter):
    '''
    Print a nicely formated table from a list of (name, shares, price, change) tuples.
    '''
    formatter.headings(['Name', 'Shares', 'Price', 'Change'])
        
    for name, shares, price, change in reportdata:
        rowdata = [ name, str(shares), f'{price:0.2f}', f'{change:0.2f}' ]
        formatter.row(rowdata)

def portfolio_report(portfoliofile, pricefile, fmt):        
    '''
    Make a stock report given portfolio and price data files.
    '''
    # Read data files 
    portfolio = read_portfolio(portfoliofile)
    prices = read_prices(pricefile)

    # Create the report data (보고서 데이터 생성)
    report = make_report_data(portfolio, prices)

    # Print it out
    formatter = tableformat.create_formatter(fmt)
    print_report(report, formatter)
    

def main(args):
    if len(args) != 4:
        raise SystemExit('Usage: %s portfile pricefile' % args[0])
    portfolio_report(args[1], args[2], args[3])

if __name__ == '__main__':
    import sys
    main(sys.argv)


Overwriting ../../test_bed/report.py


## 논의

In [ ]:
# 논의
# ----
# 라이브러리와 프레임워크에서 확장성 있는 코드를 작성하는 것은 상속의 주된 용도다. 
# 예를 들어, 프레임워크는 기본 클래스를 제공하고 당신은 그것을 상속해 객체를 정의한다. 
# 그런 다음 기능을 구현하는 다양한 메서드를 채운다.

# 또 한 가지 심오한 개념은 '추상화를 직접 작성'하는 것이다. 
# 우리는 이 연습 문제에서 테이블을 포매팅하는 클래스를 직접 정의했다. 
# 코드를 들여다보고 '포매팅 라이브러리라든지 다른 사람이 이미 만들어놓은 것을 사용해야겠다'고 생각할지 모르겠다. 
# 하지만 직접 만든 클래스와 라이브러리를 둘 다 사용해야 한다. 스스로 클래스를 만들면 커플링이 줄어들고 더 유연해진다. 
# 당신의 애플리케이션이 당신의 클래스의 프로그래밍 인터페이스를 사용하는 한, 내부 구현을 원하는 대로 변경할 수 있다. 
# 완전한 맞춤 코드를 작성할 수 있다. 다른 사람이 만든 서드 파티 패키지를 사용할 수 있다. 
# 사용하던 서드 파티 패키지보다 더 좋은 것을 찾게 되면 다른 것으로 바꿀 수 있다. 
# 패키지를 교체하는 것이 별 문제가 되지 않을 것이다. 
# 인터페이스를 유지하는 한 애플리케이션 코드가 전혀 깨지지 않을 것이기 때문이다. 
# 그것은 강력한 아이디어이며, 이와 같은 상속을 고려해야 하는 이유다.

# 객체 지향 프로그램 설계는 매우 어려울 수 있다. 
# 더 알고 싶으면 디자인 패턴을 다루는 책을 살펴보기 바란다
# (이 연습 문제에서 일어난 일을 이해하면 객체를 실용적으로 다루는 데 상당한 도움이 된다).

# 4.3 특수한 메서드

## 문자열 변환을 위한 특수 메서드(__str__, __repr__)

In [ ]:
# 특수한 메서드를 사용함으로써, 파이썬의 행위를 여러 면에서 커스터마이즈할 수 있다. 
# 이러한 특수 메서드를 "마법(magic)" 메서드라고도 부른다. 이 섹션에서 그 개념을 알아보자. 
# 동적 어트리뷰트 액세스와 바운드 메서드도 논의한다.

# 도입
# ----
# 클래스에 특수 메서드를 정의할 수 있다. 파이썬 인터프리터는 이러한 메서드들을 특별하게 다룬다. 항상 앞에 __가 붙는다. (예: __init__)

# class Stock(object):
#     def __init__(self):
#         ...
#     def __repr__(self):
#         ...
# 특수 메서드는 많이 있지만, 그중 몇 가지만 예를 들어 보자.

## 문자열 변환을 위한 특수 메서드

In [ ]:
# 문자열 변환을 위한 특수 메서드
# ----------------------------
# 객체에는 두 가지 문자열 표현이 있다.

# >>> from datetime import date
# >>> d = date(2012, 12, 21)
# >>> print(d)
# 2012-12-21

# >>> d
# datetime.date(2012, 12, 21)
# >>>


# str() 함수를 사용해 멋지게 프린트할 수 있는 출력을 생성한다.

# >>> str(d)
# '2012-12-21'
# >>>

# repr() 함수는 개발자를 위한 상세한 표현에 사용된다.

# >>> repr(d)
# 'datetime.date(2012, 12, 21)'
# >>>

# 이러한 str()과 repr() 함수는 특수한 메서드를 사용해, 클래스로부터 문자열을 생성한다.

# class Date(object):
#     def __init__(self, year, month, day):
#         self.year = year
#         self.month = month
#         self.day = day

#     # Used with `str()`
#     def __str__(self):
#         return f'{self.year}-{self.month}-{self.day}'

#     # `repr()`과 함께 사용
#     def __repr__(self):
#         return f'Date({self.year},{self.month},{self.day})'

# 참고: __repr__()이 반환하는 문자열을 eval()에 전달하면 객체를 다시 만들 수 있도록 하는 것이 관례다. 
# 그렇게 하는 것이 불가능하다면, 그 대신 읽기 쉬운 표현을 사용한다.


In [10]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    def __str__(self):
        return f'{self.year}-{self.month}-{self.day}'
    
    def __repr__(self):
        return f'Date({self.year},{self.month},{self.day})' 
    
aa = Date(2023, 3, 27)
print(aa)
       

2023-3-27


## 수학을 위한 특수 메서드

In [ ]:
# 수학을 위한 특수 메서드
# ---------------------
# 다음과 같은 메서드가 수학 연산에 사용된다.

# a + b       a.__add__(b)
# a - b       a.__sub__(b)
# a * b       a.__mul__(b)
# a / b       a.__truediv__(b)
# a // b      a.__floordiv__(b)
# a % b       a.__mod__(b)
# a << b      a.__lshift__(b)
# a >> b      a.__rshift__(b)
# a & b       a.__and__(b)
# a | b       a.__or__(b)
# a ^ b       a.__xor__(b)
# a ** b      a.__pow__(b)
# -a          a.__neg__()
# ~a          a.__invert__()
# abs(a)      a.__abs__()

## 항목 접근을 위한 특수 메서드

In [ ]:
# 항목 접근을 위한 특수 메서드
# --------------------------
# 컨테이너를 구현하는 데 다음과 같은 메서드를 사용한다.

# len(x)      x.__len__()
# x[a]        x.__getitem__(a)
# x[a] = v    x.__setitem__(a,v)
# del x[a]    x.__delitem__(a)

# 이것들을 다음과 같이 클래스에서 사용할 수 있다.

# class Sequence:
#     def __len__(self):
#         ...
#     def __getitem__(self,a):
#         ...
#     def __setitem__(self,a,v):
#         ...
#     def __delitem__(self,a):
#         ...


## 메서드 호출(. 연산자, () 연산자)

In [ ]:
# 메서드 호출
# ----------
# 메서드 호출은 두 단계 과정으로 이뤄진다.

# 메서드가 있는지 확인(lookup): . 연산자
# 메서드 호출: () 연산자

# >>> s = Stock('GOOG',100,490.10)
# >>> c = s.cost  # 메서드 확인
# >>> c
# <bound method Stock.cost of <Stock object at 0x590d0>>

# >>> c()         # 메서드 호출
# 49010.0
# >>>

In [13]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
    
    def __str__(self):
        return f'{self.year}-{self.month}-{self.day}'
    
    def __repr__(self):
        return f'Date({self.year},{self.month},{self.day})' 
    
    def print_date(self):
        print(self.year)
    
d = Date(2023, 3, 27)
e = d.print_date
e
e()
       

2023


## 바운드 메서드

In [ ]:
# 바운드 메서드
# ------------
# 함수 호출 연산자 ()에 의해 호출되지 않은 메서드를 바운드 메서드(bound method)라 한다. 
# 바운드 메서드는 그것이 어느 인스턴스에서 왔는지를 반환한다.

# >>> s = Stock('GOOG', 100, 490.10)
# >>> s
# <Stock object at 0x590d0>
# >>> c = s.cost
# >>> c
# <bound method Stock.cost of <Stock object at 0x590d0>>
# >>> c()
# 49010.0
# >>>
# 바운드 메서드는 부주의하고 명백하지 않은 오류의 원인이 되곤 한다. 예:

# >>> s = Stock('GOOG', 100, 490.10)
# >>> print('Cost : %0.2f' % s.cost)
# Traceback (most recent call last):
#   File "<stdin>", line 1, in <module>
# TypeError: float argument required
# >>>
# 또는 디버깅하기 힘든 잘못된 행동을 한다.

# f = open(filename, 'w')
# ...
# f.close     # 아차, 아무 일도 하지 않았다. `f`는 여전히 열려 있다.
# 두 경우 모두 괄호를 붙이는 것을 잊었기 때문에 오류가 발생했다. 예: s.cost() 또는 f.close().


In [26]:
# %load ../../test_bed/stock.py
class Stock:
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price
        
    def cost(self):
        return self.shares * self.price
        
    def sell(self, shares):
        self.shares -= shares            

s = Stock('GOOG', 100, 490.10)
isinstance(s, Stock) 
s     
c = s.cost  
c()

s = Stock('GOOG', 100, 490.10)
#print('Cost : %0.2f' % s.cost)

f = open('../../data/portfolio.csv', 'r')
#f.close

In [ ]:

# 어트리뷰트 액세스
# 어트리뷰트에 액세스, 조작, 관리하는 또 다른 방법이 있다.

# getattr(obj, 'name')          # obj.name과 같음
# setattr(obj, 'name', value)   # obj.name = value와 같음
# delattr(obj, 'name')          # del obj.name과 같음
# hasattr(obj, 'name')          # 어트리뷰트가 존재하는지 테스트
# 예:

# if hasattr(obj, 'x'):
#     x = getattr(obj, 'x'):
# else:
#     x = None
# 참고: getattr()에도 유용한 기본값 arg가 있다.

# x = getattr(obj, 'x', None)
# 연습 문제